In [2]:
import pandas as pd

# Caminho do arquivo
DATA_PATH = "/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw/brasileirao_serie_a_2003_2024.csv"

# Leitura
df = pd.read_csv(DATA_PATH)

# Visão geral
display(df.head())
print("Shape:", df.shape)
print("\nColunas:")
print(df.columns)


,ano_campeonato,data,rodada,estadio,arbitro,publico,publico_max,time_mandante,time_visitante,tecnico_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2005,2005-10-08,31,Estádio Durival Britto e Silva,NaN,NaN,NaN,Paraná,Fluminense,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,2005-04-23,1,Estádio Durival Britto e Silva,NaN,NaN,NaN,Paraná,Goiás EC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,2005-08-21,21,Estádio Durival Britto e Silva,NaN,NaN,NaN,Paraná,Vasco da Gama,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,2005-08-28,23,Estádio Durival Britto e Silva,NaN,NaN,NaN,Paraná,São Paulo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,2005-06-26,9,Estádio Olímpico Nilton Santos,NaN,NaN,NaN,Botafogo,Figueirense FC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape: (8453, 35)

Colunas:
Index(['ano_campeonato', 'data', 'rodada', 'estadio', 'arbitro', 'publico',
       'publico_max', 'time_mandante', 'time_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'colocacao_mandante', 'colocacao_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante'],
      dtype='object')


In [3]:
df.info()
df.isnull().mean().sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ano_campeonato                  8453 non-null   int64  
 1   data                            8453 non-null   object 
 2   rodada                          8453 non-null   int64  
 3   estadio                         8442 non-null   object 
 4   arbitro                         6743 non-null   object 
 5   publico                         6742 non-null   float64
 6   publico_max                     4181 non-null   float64
 7   time_mandante                   8453 non-null   object 
 8   time_visitante                  8453 non-null   object 
 9   tecnico_mandante                6299 non-null   object 
 10  tecnico_visitante               6299 non-null   object 
 11  colocacao_mandante              6743 non-null   float64
 12  colocacao_visitante             67

,0
chutes_fora_visitante,0.753105
defesas_visitante,0.753105
defesas_mandante,0.753105
faltas_mandante,0.753105
faltas_visitante,0.753105
chutes_bola_parada_visitante,0.753105
chutes_fora_mandante,0.753105
impedimentos_visitante,0.753105
impedimentos_mandante,0.753105
chutes_visitante,0.753105


In [8]:
df["data"] = pd.to_datetime(df["data"], errors="coerce")
df = df.sort_values("data").reset_index(drop=True)


In [9]:
df["data"].dtype


dtype('<M8[ns]')

In [10]:
df[["data", "ano_campeonato", "rodada"]].head(5)


,data,ano_campeonato,rodada
0,2003-03-29,2003,1
1,2003-03-29,2003,1
2,2003-03-30,2003,1
3,2003-03-30,2003,1
4,2003-03-30,2003,1


In [11]:
df[["data", "ano_campeonato", "rodada"]].tail(5)


,data,ano_campeonato,rodada
8448,2024-10-26,2024,31
8449,2024-10-26,2024,31
8450,2024-10-26,2024,31
8451,2024-10-26,2024,31
8452,2024-10-26,2024,31


In [12]:
df["data"].isna().sum()


np.int64(0)

In [13]:
# Filtrar apenas temporada 2024
df_2024 = df[df["ano_campeonato"] == 2024]

# Total de rodadas distintas
df_2024["rodada"].nunique()


29

In [14]:
sorted(df_2024["rodada"].unique())


[np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(8),
 np.int64(9),
 np.int64(12),
 np.int64(13),
 np.int64(14),
 np.int64(15),
 np.int64(16),
 np.int64(17),
 np.int64(18),
 np.int64(19),
 np.int64(20),
 np.int64(21),
 np.int64(22),
 np.int64(23),
 np.int64(24),
 np.int64(25),
 np.int64(26),
 np.int64(27),
 np.int64(28),
 np.int64(29),
 np.int64(30),
 np.int64(31)]

In [15]:
df_2024.groupby("rodada").size()


,0
rodada,
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10


In [17]:
def get_result(row):
    if row["gols_mandante"] > row["gols_visitante"]:
        return "H"
    elif row["gols_mandante"] < row["gols_visitante"]:
        return "A"
    else:
        return "D"

df["resultado"] = df.apply(get_result, axis=1)

# Conferência
df["resultado"].value_counts()


,count
resultado,
H,4182
D,2249
A,2022


In [18]:
df.columns


Index(['ano_campeonato', 'data', 'rodada', 'estadio', 'arbitro', 'publico',
       'publico_max', 'time_mandante', 'time_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'colocacao_mandante', 'colocacao_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante',
       'resultado'],
      dtype='object')

In [19]:
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim")
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Garantir datetime e ordenação (se já fez, ok repetir)
import pandas as pd
df["data"] = pd.to_datetime(df["data"], errors="coerce")
df = df.sort_values("data").reset_index(drop=True)

cbf_base = df[df["ano_campeonato"] <= 2023][
    ["ano_campeonato", "data", "time_mandante", "time_visitante",
     "gols_mandante", "gols_visitante", "resultado"]
].dropna().copy()

print("CBF base shape:", cbf_base.shape)
print("Anos:", cbf_base["ano_campeonato"].min(), "→", cbf_base["ano_campeonato"].max())

out_path = BASE_DIR / "cbf_2003_2023.parquet"
cbf_base.to_parquet(out_path, index=False)

print("✅ salvo:", out_path)


CBF base shape: (8169, 7)
Anos: 2003 → 2023
✅ salvo: /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim/cbf_2003_2023.parquet
